In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import csv
from sklearn.preprocessing import add_dummy_feature,OrdinalEncoder
from sklearn.feature_selection import SelectKBest,RFE
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
import warnings
from sklearn import preprocessing
from mlxtend.classifier import StackingCVClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn import model_selection
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression


In [21]:
### Reading the Train event data
df_events_with = pd.read_csv("train_with_event.csv", dtype={'device_id': np.str, 'latitude': np.float, 'longitude':np.float,'event_id': np.str})
### Reading the APP Event Data 
df_events_without = pd.read_csv("train_without_event.csv", dtype={'device_id': np.str, 'latitude': np.float, 'longitude':np.float,'event_id': np.str})


In [22]:
df_events_with.head()

,Unnamed: 0,device_id,gender,age,group_train,event_id,datetimestamp,latitude,longitude,event_day,...,latitude_difference,longitude_difference,latitude_medium,longitude_medium,cluster_label,app_id,is_installed,is_active,phone_brand,category
0,0,-7548291590301750000,M,33,M32+,1141870,2016-05-01 10:08:16,33.98,116.79,1.0,...,0.0,0.0,33.98,116.79,-1,3433289601737013244,1,1,Huawei,Industry
1,2,7442042493953950000,M,26,M25-32,2591882,2016-05-01 18:17:43,39.43,116.98,1.0,...,0.0,0.0,39.43,116.98,-1,3433289601737013244,1,1,Huawei,Industry
2,4,-800490591774117000,F,33,F32+,1455673,2016-05-01 16:45:07,0.00,0.00,1.0,...,0.0,0.0,0.00,0.00,-1,3433289601737013244,1,1,Huawei,Industry
3,6,-7430262216439700000,M,42,M32+,2584141,2016-05-07 12:00:09,22.51,113.92,7.0,...,0.0,0.0,22.51,113.92,9,3433289601737013244,1,1,Huawei,Industry
4,8,-7430262216439700000,M,42,M32+,2136511,2016-05-02 23:33:34,22.51,113.92,2.0,...,0.0,0.0,22.51,113.92,9,3433289601737013244,1,1,Huawei,Industry


###  Create the Age Group as Label for output

In [23]:
# Define the age groups
bins = [0, 24, 32, 45, 150]
labels = ['0-24', '25-32', '33-45', '46+']

# Categorize the ages into groups
df_events_with['age_group'] = pd.cut(df_events_with['age'], bins=bins, labels=labels, right=False)

In [24]:
df_events_with.head()

,Unnamed: 0,device_id,gender,age,group_train,event_id,datetimestamp,latitude,longitude,event_day,...,longitude_difference,latitude_medium,longitude_medium,cluster_label,app_id,is_installed,is_active,phone_brand,category,age_group
0,0,-7548291590301750000,M,33,M32+,1141870,2016-05-01 10:08:16,33.98,116.79,1.0,...,0.0,33.98,116.79,-1,3433289601737013244,1,1,Huawei,Industry,33-45
1,2,7442042493953950000,M,26,M25-32,2591882,2016-05-01 18:17:43,39.43,116.98,1.0,...,0.0,39.43,116.98,-1,3433289601737013244,1,1,Huawei,Industry,25-32
2,4,-800490591774117000,F,33,F32+,1455673,2016-05-01 16:45:07,0.00,0.00,1.0,...,0.0,0.00,0.00,-1,3433289601737013244,1,1,Huawei,Industry,33-45
3,6,-7430262216439700000,M,42,M32+,2584141,2016-05-07 12:00:09,22.51,113.92,7.0,...,0.0,22.51,113.92,9,3433289601737013244,1,1,Huawei,Industry,33-45
4,8,-7430262216439700000,M,42,M32+,2136511,2016-05-02 23:33:34,22.51,113.92,2.0,...,0.0,22.51,113.92,9,3433289601737013244,1,1,Huawei,Industry,33-45


In [6]:
df_events_with.columns

Index(['Unnamed: 0', 'device_id', 'gender', 'age', 'group_train', 'event_id',
       'datetimestamp', 'latitude', 'longitude', 'event_day', 'event_avg',
       'latitude_difference', 'longitude_difference', 'latitude_medium',
       'longitude_medium', 'cluster_label', 'app_id', 'is_installed',
       'is_active', 'phone_brand', 'category', 'age_group'],
      dtype='object')

### Drop the column which is require 

In [25]:
df_events_with.drop(['Unnamed: 0','event_id','age','group_train','datetimestamp','latitude','longitude','app_id','event_day'],axis= 1 , inplace=True)

In [26]:
df_events_with.columns

Index(['device_id', 'gender', 'event_avg', 'latitude_difference',
       'longitude_difference', 'latitude_medium', 'longitude_medium',
       'cluster_label', 'is_installed', 'is_active', 'phone_brand', 'category',
       'age_group'],
      dtype='object')

In [27]:
df_events_with.shape

(537737, 13)

In [28]:
df_encoded = pd.get_dummies(df_events_with, columns=['phone_brand', 'category'])


In [29]:
df_encoded.head()

,device_id,gender,event_avg,latitude_difference,longitude_difference,latitude_medium,longitude_medium,cluster_label,is_installed,is_active,...,category_Industry,category_Nature,category_Other,category_Personal Effectiveness,category_Property,category_Relatives,category_Religion,category_Science and Technology,category_Services,category_Shopping
0,-7548291590301750000,M,0.154110,0.0,0.0,33.98,116.79,-1,1,1,...,1,0,0,0,0,0,0,0,0,0
1,7442042493953950000,M,1.000000,0.0,0.0,39.43,116.98,-1,1,1,...,1,0,0,0,0,0,0,0,0,0
2,-800490591774117000,F,1.000000,0.0,0.0,0.00,0.00,-1,1,1,...,1,0,0,0,0,0,0,0,0,0
3,-7430262216439700000,M,0.228571,0.0,0.0,22.51,113.92,9,1,1,...,1,0,0,0,0,0,0,0,0,0
4,-7430262216439700000,M,0.028571,0.0,0.0,22.51,113.92,9,1,1,...,1,0,0,0,0,0,0,0,0,0


In [30]:
train_test_data = pd.read_csv('train_test_split.csv',dtype={'device_id': np.str})

In [31]:
train_test_data.dtypes

device_id          object
gender             object
age                 int64
group              object
train_test_flag    object
dtype: object

In [32]:
df_encoded = df_encoded.merge(train_test_data[['device_id','train_test_flag']], on='device_id')


In [33]:
train_data = df_encoded[df_encoded['train_test_flag'] == 'train']
test_data = df_encoded[df_encoded['train_test_flag'] == 'test']

In [34]:
print("Shape of training data",train_data.shape)
print("Shape of test data",test_data.shape)

Shape of training data (403016, 38)
Shape of test data (134721, 38)


In [ ]:
train_data.to_csv('traindata.csv')
test_data.to_csv('testdata.csv')

In [25]:
def age_group(age):
    if age == '0-24':
        return 0
    elif age == '25-32':
        return 1
    elif age == '33-45':
        return 2
    else:
        return 3

In [16]:
train_data.columns

Index(['device_id', 'gender', 'event_avg', 'latitude_difference',
       'longitude_difference', 'latitude_medium', 'longitude_medium',
       'cluster_label', 'is_installed', 'is_active', 'category', 'age_group',
       'phone_brand_Coolpad', 'phone_brand_Gionee', 'phone_brand_Huawei',
       'phone_brand_Meizu', 'phone_brand_OPPO', 'phone_brand_Xiaomi',
       'phone_brand_lenovo', 'phone_brand_other', 'phone_brand_samsung',
       'phone_brand_vivo', 'phone_brand_Coolpad', 'phone_brand_Gionee',
       'phone_brand_Huawei', 'phone_brand_Meizu', 'phone_brand_OPPO',
       'phone_brand_Xiaomi', 'phone_brand_lenovo', 'phone_brand_other',
       'phone_brand_samsung', 'phone_brand_vivo', 'train_test_flag'],
      dtype='object')

### Target Encoding for age_group and Gender

In [26]:
train_data['age_group'] = train_data['age_group'].apply(lambda x : age_group(x) )
test_data['age_group'] = test_data['age_group'].apply(lambda x: age_group(x))
train_data['gender'] = train_data['gender'].apply(lambda x: 0 if x == 'M' else 1)
test_data['gender'] = test_data['gender'].apply(lambda x: 0 if x=='M' else 1)

In [28]:
age_train = train_data['age_group'].values
gender_train = train_data['gender'].values
age_test = test_data['age_group'].values
gender_test = test_data['age_group'].values
X_train = train_data.drop(['device_id','gender','train_test_flag','age_group'],axis = 1)
X_test = train_data.drop(['device_id','gender','train_test_flag','age_group'],axis = 1)

### Creating the machine learning model